# Winner of the tournament
We add columns to tournamen_info_df that have:
- The winner of the tournament
- If the winner came from losers
- The ratings of players playing in the top 8

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import datetime 

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, cohen_kappa_score
from sklearn.model_selection import train_test_split  # Correct import

import sqlite3
import sys
import time
import tqdm
from tqdm.auto import tqdm
import pickle
import joblib
import os

if os.path.exists('/workspace/data_2'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data_2/'
else:
    data_path = '../data/'


### Load Data


In [2]:
sets_df = pd.read_pickle(data_path + 'full_dataset_df.pkl')

sets_df.head(3)



,key_x,game,tournament_key,winner_id,loser_id,p1_id,p2_id,p1_score,p2_score,valid_score,...,p1/m1/m1_alt2_updates,p2/m1/m1_alt2_elo,p2/m1/m1_alt2_rd,p2/m1/m1_alt2_updates,p1/m1_alt3_elo,p1/m1_alt3_rd,p1/m1_alt3_updates,p2/m1_alt3_elo,p2/m1_alt3_rd,p2/m1_alt3_updates
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,Chillin,Chillin,5620,1,3,True,...,0.0,1951.007767,92.862909,6.0,1500.0,350.0,0.0,1613.001275,70.600790,5.0
1,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,Chillin,Chillin,5620,1,3,True,...,0.0,1951.007767,92.862909,6.0,1500.0,350.0,0.0,1613.001275,70.600790,5.0
2,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,15634,3,2,True,...,0.0,2217.315033,196.486995,4.0,1500.0,350.0,0.0,1637.162591,77.931512,5.0


In [3]:
for i, col in enumerate(sets_df.columns):
    print(i, col)

0 key_x
1 game
2 tournament_key
3 winner_id
4 loser_id
5 p1_id
6 p2_id
7 p1_score
8 p2_score
9 valid_score
10 best_of
11 location_names
12 bracket_name
13 bracket_order
14 set_order
15 game_data
16 top_8
17 top_8_location_names
18 valid_top_8_bracket
19 top_8_bracket_location_names
20 major
21 key_y
22 start
23 end
24 start_week
25 p1_characters
26 p2_characters
27 p1_consistent
28 p2_consistent
29 matchup_strings
30 end_week
31 players_have_history
32 (p1/p2)_sorted
33 (p1/p2)_was_sorted
34 results_sorted
35 results
36 matchup_1
37 matchup_2
38 matchup_3
39 matchup_4
40 matchup_5
41 matchup_6
42 matchup_7
43 matchup_8
44 matchup_9
45 matchup_10
46 winner
47 p1_default_elo
48 p2_default_elo
49 p1_default_rd
50 p2_default_rd
51 p1_default_updates
52 p2_default_updates
53 start_index
54 start_date
55 p1_fox_count
56 p1_falco_count
57 p1_marth_count
58 p1_sheik_count
59 p1_captainfalcon_count
60 p1_jigglypuff_count
61 p1_peach_count
62 p1_luigi_count
63 p1_samus_count
64 p1_ganondorf_coun

In [43]:
tournament_info_df = pd.read_pickle(data_path + 'top_8_tournament_previous_sets_and_results_df')
print(tournament_info_df.columns)
tournament_info_df.head()

Index(['game', 'key', 'cleaned_name', 'source', 'tournament_name',
       'tournament_event', 'season', 'rank', 'start', 'end', 'country',
       'state', 'city', 'entrants', 'placings', 'losses', 'bracket_types',
       'online', 'lat', 'lng', 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
       'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2',
       'LN_A_p1_non_top_8_sets', 'LN_A_p2_non_top_8_sets',
       'LN_B_p1_non_top_8_sets', 'LN_B_p2_non_top_8_sets',
       'WSF_A_p1_non_top_8_sets', 'WSF_A_p2_non_top_8_sets',
       'WSF_B_p1_non_top_8_sets', 'WSF_B_p2_non_top_8_sets'],
      dtype='object')


,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,...,WSF_B_p1,WSF_B_p2,LN_A_p1_non_top_8_sets,LN_A_p2_non_top_8_sets,LN_B_p1_non_top_8_sets,LN_B_p2_non_top_8_sets,WSF_A_p1_non_top_8_sets,WSF_A_p2_non_top_8_sets,WSF_B_p1_non_top_8_sets,WSF_B_p2_non_top_8_sets
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,...,Ginger,1008,"[(84, True), (110, True), (123, False), (210, ...","[(77, True), (107, True), (122, False), (209, ...","[(85, True), (111, True), (124, True), (130, F...","[(92, True), (114, True), (125, False), (204, ...","[(81, True), (109, True), (123, True), (130, T...","[(73, True), (105, True), (121, True), (129, T...","[(97, True), (117, True), (127, True), (132, T...","[(89, True), (113, True), (125, True), (131, T..."
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print(tournament_info_df.head(2).to_string())
print()
mask = (tournament_info_df['key']=='s@sh7')
print(tournament_info_df.loc[mask].to_string())

    game                                        key                               cleaned_name     source                               tournament_name tournament_event season rank               start                 end country state           city  entrants                                                                                                               placings losses bracket_types  online  lat  lng LN_A_p1 LN_A_p2 LN_B_p1        LN_B_p2 WSF_A_p1           WSF_A_p2 WSF_B_p1 WSF_B_p2                                                          LN_A_p1_non_top_8_sets                                                           LN_A_p2_non_top_8_sets                                                                        LN_B_p1_non_top_8_sets                                                                        LN_B_p2_non_top_8_sets                               WSF_A_p1_non_top_8_sets                               WSF_A_p2_non_top_8_sets                               WSF_B_p1_n

In [31]:
top_8_columns = ['key', 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2', 
                 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']
print(tournament_info_df[top_8_columns].head().to_string())

                                             key LN_A_p1 LN_A_p2 LN_B_p1        LN_B_p2 WSF_A_p1           WSF_A_p2 WSF_B_p1 WSF_B_p2
0      mdva-invitational-2017-(challonge-mirror)    math    lain      tm  Bbbbbbbbbexic      kjh  Mew2king (unpaid)   Ginger     1008
1                                          s@sh7    1008    1023   13932           1004     1000              16342     1003     4465
2  slippi-champions-league-week-1__melee-singles    Hack   Vdogg    4215         Thomas     6103               6189     4465    12870
3  slippi-champions-league-week-2__melee-singles    Prof   48789   12870           5956    19573               1000     4465     6189
4  slippi-champions-league-week-3__melee-singles   16342    4465    1008           1013     1004               6189     1003    15990


In [42]:
valid_sets_df = sets_df[sets_df['valid_top_8_bracket']==True].copy()
valid_sets_df

,key_x,game,tournament_key,winner_id,loser_id,p1_id,p2_id,p1_score,p2_score,valid_score,...,p1/m1/m1_alt2_updates,p2/m1/m1_alt2_elo,p2/m1/m1_alt2_rd,p2/m1/m1_alt2_updates,p1/m1_alt3_elo,p1/m1_alt3_rd,p1/m1_alt3_updates,p2/m1_alt3_elo,p2/m1_alt3_rd,p2/m1_alt3_updates
3579,90101116,melee,s@sh7,Mew2king (unpaid),kjh,kjh,Mew2king (unpaid),0,1,False,...,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0
3580,90101117,melee,s@sh7,Ginger,1008,Ginger,1008,1,0,False,...,0.0,2038.035631,55.022254,15.0,1500.000000,350.000000,0.0,1723.326663,45.482726,23.0
3581,90101118,melee,s@sh7,Mew2king (unpaid),Ginger,Mew2king (unpaid),Ginger,1,0,False,...,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0
7600,90101203,melee,s@sh7,tm,Bbbbbbbbbexic,tm,Bbbbbbbbbexic,1,0,False,...,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0
7601,90101204,melee,s@sh7,lain,math,math,lain,0,1,False,...,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0,1500.000000,350.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13804772,gg__76279646,melee,dice-dance-17__1v1-melee,908884,3196854,908884,3196854,3,1,False,...,8.0,1367.847397,71.343534,12.0,1382.928183,44.319944,18.0,1359.333645,46.757059,19.0
13804773,gg__76279647,melee,dice-dance-17__1v1-melee,2791218,495503,2791218,495503,3,1,False,...,0.0,1253.079472,89.514978,10.0,1500.000000,350.000000,0.0,1320.293688,45.605920,27.0
13804774,gg__76279648,melee,dice-dance-17__1v1-melee,908884,2407110,2407110,908884,0,3,False,...,6.0,1431.762802,76.078492,16.0,1033.288619,82.850548,6.0,1382.928183,44.319944,18.0
13804775,gg__76279649,melee,dice-dance-17__1v1-melee,2791218,908884,908884,2791218,0,3,False,...,11.0,1500.000000,350.000000,0.0,1382.928183,44.319944,18.0,1500.000000,350.000000,0.0


### Winner of the tournament

In [44]:
valid_sets = sets_df[sets_df['valid_top_8_bracket']==True]

# Identify tournaments with 'GFR'
GFR_tournament_keys = valid_sets.loc[(valid_sets['top_8_bracket_location_names'] == 'GFR'), 'tournament_key'].copy()

# Filter 'GF' tournaments that do not have a corresponding 'GFR'
GF_tournament_results_df = valid_sets.loc[
    (valid_sets['top_8_bracket_location_names'] == 'GF') & (~valid_sets['tournament_key'].isin(GFR_tournament_keys)),
    ['tournament_key', 'winner_id']
].copy()

GF_tournament_results_df['GFR'] = False

# Filter 'GFR' tournament results
GFR_tournament_results_df = valid_sets.loc[
    valid_sets['top_8_bracket_location_names'] == 'GFR',
    ['tournament_key', 'winner_id']
]
GFR_tournament_results_df['GFR'] = True

# Concatenate the results
tournament_results_df = pd.concat([GF_tournament_results_df, GFR_tournament_results_df], ignore_index=False)

# View the results
tournament_results_df.head()


,tournament_key,winner_id,GFR
7606,s@sh7,Mew2king (unpaid),False
7998,httpsparagonchallongecomla_2015_melee_singles,1000,False
9392,httpsdl4-5challongecomdl45meleeprobracket,6189,False
12106,httpsbeastsmashchallongecomb5msb,4465,False
15774,httpsparagonchallongecomorlando_2015_melee_sin...,1004,False


### Did the winner of the tournament come from winners side?

In [48]:
    # winners_finals_players_df = sets_df[(sets_df['top_8']==True) & (sets_df['top_8_location_names']=='WF')][['tournament_key', 'p1_id', 'p2_id']]
losers_finals_players_df = valid_sets[(valid_sets['top_8']==True) & (valid_sets['top_8_location_names']=='LF')][['tournament_key', 'p1_id', 'p2_id']]

# losers_finals_players_df.rename(columns={'p1_id': 'p1_id', 'p2_id': 'GF_p2_id'}, inplace=True)

tournament_results_df['from_WF'] = True

# Create a mapping of tournament_key to all players in losers finals
losers_finals_players_dict = (
    losers_finals_players_df
    .groupby('tournament_key')[['p1_id', 'p2_id']]
    .apply(lambda df: set(df.values.flatten()))
    .to_dict()
)

# Update 'from_WF' column based on whether the winner_id is in the losers finals players
tournament_results_df['from_WF'] = tournament_results_df.apply(
    lambda row: row['winner_id'] in losers_finals_players_dict.get(row['tournament_key'], set()),
    axis=1
)


tournament_results_df.head()

,tournament_key,winner_id,GFR,from_WF
7606,s@sh7,Mew2king (unpaid),False,False
7998,httpsparagonchallongecomla_2015_melee_singles,1000,False,False
9392,httpsdl4-5challongecomdl45meleeprobracket,6189,False,False
12106,httpsbeastsmashchallongecomb5msb,4465,False,False
15774,httpsparagonchallongecomorlando_2015_melee_sin...,1004,False,False


In [ ]:
result = pd.merge(
    tournament_info_df,
    tournament_results_df[['tournament_key', 'winner_id']],
    how='left',
    left_on='key',
    right_on='tournament_key'
)

# Drop the 'tournament_key' column
result = result.drop(columns=['tournament_key'])

# Display the result
result.head()

result.to_pickle(data_path + 'top_8_tournament_previous_sets_and_results_with_winners_df')

,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,...,WSF_B_p2,LN_A_p1_non_top_8_sets,LN_A_p2_non_top_8_sets,LN_B_p1_non_top_8_sets,LN_B_p2_non_top_8_sets,WSF_A_p1_non_top_8_sets,WSF_A_p2_non_top_8_sets,WSF_B_p1_non_top_8_sets,WSF_B_p2_non_top_8_sets,winner_id
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,...,1008,"[(84, True), (110, True), (123, False), (210, ...","[(77, True), (107, True), (122, False), (209, ...","[(85, True), (111, True), (124, True), (130, F...","[(92, True), (114, True), (125, False), (204, ...","[(81, True), (109, True), (123, True), (130, T...","[(73, True), (105, True), (121, True), (129, T...","[(97, True), (117, True), (127, True), (132, T...","[(89, True), (113, True), (125, True), (131, T...",Mew2king (unpaid)
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# print(f"The winner of the tournament came from winners side {tournament_results_df['from_winners'].sum() / tournament_results_df.shape[0]:.0%} of the time.")
print(f"The winner of the tournament came from winners finals {tournament_results_df['from_WF'].sum() / tournament_results_df.shape[0]:.0%} of the time.")
print(f"A grand finals reset happened {tournament_results_df['GFR'].sum() / tournament_results_df.shape[0]:.0%} of the time.")
print(f"Given that a grand final reset occured, the player coming from the losers finals won {(1-tournament_results_df['from_WF']).sum() / tournament_results_df['GFR'].sum():.0%} of the time.")

The winner of the tournament came from winners finals 18% of the time.
A grand finals reset happened 31% of the time.
Given that a grand final reset occured, the player coming from the losers finals won 260% of the time.


In [52]:
top_8_player_columns = ['key', 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
                        'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']

# Perform the merge and retain the index of `tournament_results_df`
merged_df = pd.merge(
    tournament_results_df,
    tournament_info_df[top_8_player_columns],
    how='left',
    left_on='tournament_key',
    right_on='key'
)

# Drop the 'key' column
merged_df = merged_df.drop(columns=['key'])

# Preserve the original index of `tournament_results_df`
merged_df.index = tournament_results_df.index

# Update 'from_winners' column based on whether the winner_id is in the WSF player columns
merged_df['from_winners'] = merged_df.apply(
    lambda row: row['winner_id'] in [
        row['WSF_A_p1'], row['WSF_A_p2'], row['WSF_B_p1'], row['WSF_B_p2']
    ],
    axis=1
)

# View the resulting DataFrame
print(merged_df.head())


                                          tournament_key          winner_id  \
7606                                               s@sh7  Mew2king (unpaid)   
7998       httpsparagonchallongecomla_2015_melee_singles               1000   
9392           httpsdl4-5challongecomdl45meleeprobracket               6189   
12106                   httpsbeastsmashchallongecomb5msb               4465   
15774  httpsparagonchallongecomorlando_2015_melee_sin...               1004   

         GFR  from_WF LN_A_p1 LN_A_p2 LN_B_p1        LN_B_p2 WSF_A_p1  \
7606   False    False    math    lain      tm  Bbbbbbbbbexic      kjh   
7998   False    False    1008    1023   13932           1004     1000   
9392   False    False    Hack   Vdogg    4215         Thomas     6103   
12106  False    False    Prof   48789   12870           5956    19573   
15774  False    False   16342    4465    1008           1013     1004   

                WSF_A_p2 WSF_B_p1 WSF_B_p2  from_winners  
7606   Mew2king (unpaid)   